In [1]:
import requests
import json
import csv
from bs4 import BeautifulSoup

In [2]:
def tratar_strings(string, type_string):
    if type_string == 'STR':
        if string != None:
            return str(string.text).replace('\n','').replace('\t','').replace('\r','').replace(';','').strip()
        else:
            return ''
    elif type_string == 'URL':
        return str(string['href']).strip()
    elif type_string == 'STR_AVAL':
        if string != None:
            return str(string['data-tooltip-html']).replace('<br>','.').strip()
        else:
            return ''

In [5]:
def get_game_detail(obj_bs4):
    json_jogo = {
        'nome': '',
        'url' : '',
        'data_lancamento' : '',
        'avaliacoes' : '',
        'valor_inicial' : '',
        'porcentagem_desconto' : '',
        'valor_final' : ''
    }
    
    json_jogo['nome'] = tratar_strings(
        obj_bs4.select_one('div.col.search_name.ellipsis'), 'STR'
    )
    
    json_jogo['url'] = tratar_strings(
        obj_bs4.select_one('a'), 'URL'
    )
    
    json_jogo['data_lancamento'] = tratar_strings(
        obj_bs4.select_one('div.col.search_released.responsive_secondrow'), 'STR'
    )
    
    json_jogo['avaliacoes'] = tratar_strings(
        obj_bs4.select_one('span.search_review_summary'), 'STR_AVAL'
    )
    
    json_jogo['valor_inicial'] = tratar_strings(
        obj_bs4.select_one('div.col.search_price.discounted.responsive_secondrow span strike'), 'STR'
    )
    
    json_jogo['porcentagem_desconto'] = tratar_strings(
        obj_bs4.select_one('div.col.search_discount.responsive_secondrow span'), 'STR'
    )

    json_jogo['valor_final'] = tratar_strings(
        obj_bs4.select_one('div.col.search_price.discounted.responsive_secondrow'), 'STR'
    ).replace(json_jogo['valor_inicial'], '')
    
    return json_jogo

In [6]:
def get_all_games(url):
    lista_jogos = []
    
    requisicao = requests.get(url)
    json_retorno = json.loads(requisicao.content.decode('utf-8'))
    obj_bs4 = BeautifulSoup(json_retorno['results_html'])
    
    for bloco_jogo in obj_bs4.select('a.search_result_row'):
        obj_jogo = BeautifulSoup(str(bloco_jogo), 'html.parser')
        
        jogo = get_game_detail(obj_jogo)
        lista_jogos.append(
            jogo
        )
    
    return lista_jogos, json_retorno['total_count']

In [7]:
lista_final_jogos = []

initial = 0
final = 100
while True:
    print ('Total de Jogos : ', len(lista_final_jogos))
    lista_jogos, final = get_all_games(
        f'https://store.steampowered.com/search/results/?query&start={initial}&count={final}&dynamic_data=&sort_by=_ASC&specials=1&filter=topsellers&infinite=1'
    )
    
    lista_final_jogos+=lista_jogos
    initial+=100
    
    if initial > final:
        break

Total de Jogos :  0
Total de Jogos :  100
Total de Jogos :  200
Total de Jogos :  300
Total de Jogos :  400
Total de Jogos :  500
Total de Jogos :  600
Total de Jogos :  700
Total de Jogos :  800
Total de Jogos :  900
Total de Jogos :  1000
Total de Jogos :  1100
Total de Jogos :  1200
Total de Jogos :  1300
Total de Jogos :  1400


In [8]:
for jogos in lista_final_jogos:
    print ('Nome do jogo : {}\nLink do Jogo : {}\nData do Lançamento : {}\nAvaliações : {}\nValor Inicial : {}\nPocentagem de desconto : {}\nValor Final : {}\n\n'.format(
        jogos['nome'],
        jogos['url'],
        jogos['data_lancamento'],
        jogos['avaliacoes'],
        jogos['valor_inicial'],
        jogos['porcentagem_desconto'],
        jogos['valor_final'],
    ))

Nome do jogo : Outer Wilds - Echoes of the Eye
Link do Jogo : https://store.steampowered.com/app/1622100/Outer_Wilds__Echoes_of_the_Eye/?snr=1_7_7_2300_150_1
Data do Lançamento : 28 Sep, 2021
Avaliações : Very Positive.97% of the 175 user reviews for this game are positive.
Valor Inicial : R$ 31,79
Pocentagem de desconto : -10%
Valor Final : R$ 28,61


Nome do jogo : Need for Speed™ Heat
Link do Jogo : https://store.steampowered.com/app/1222680/Need_for_Speed_Heat/?snr=1_7_7_2300_150_1
Data do Lançamento : 4 Jun, 2020
Avaliações : Very Positive.84% of the 24,559 user reviews for this game are positive.
Valor Inicial : R$ 279,00
Pocentagem de desconto : -86%
Valor Final : R$ 39,06


Nome do jogo : Outer Wilds
Link do Jogo : https://store.steampowered.com/app/753640/Outer_Wilds/?snr=1_7_7_2300_150_1
Data do Lançamento : 18 Jun, 2020
Avaliações : Overwhelmingly Positive.95% of the 21,855 user reviews for this game are positive.
Valor Inicial : R$ 47,49
Pocentagem de desconto : -40%
Valor 

In [9]:
with open('jogos.csv', 'w', encoding='utf-8') as csv_file:
    csvWriter = csv.writer(csv_file, delimiter=';')
    
    csvWriter.writerow([
        'nome',
        'url',
        'data_lancamento',
        'avaliacoes',
        'valor_inicial',
        'porcentagem_desconto',
        'valor_final'
    ])
    
    for jogos in lista_final_jogos:
        csvWriter.writerow([
            jogos['nome'],
            jogos['url'],
            jogos['data_lancamento'],
            jogos['avaliacoes'],
            jogos['valor_inicial'],
            jogos['porcentagem_desconto'],
            jogos['valor_final']
        ])